In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification 
import pandas as pd
import librosa
import numpy as np
from tqdm import tqdm

import sys
sys.path.append("..")
from src.models import EModel, Wav2Vec2Facebook
from src.datamodules import Wav2Vec2DataModule

In [2]:
class Config:
    seed = 42
    batch_size=32
    
config = Config()

datamodule = Wav2Vec2DataModule(train_csv="../dataset/train.csv", test_csv="../dataset/test.csv", config=config)
datamodule.setup(stage='test')
test_loader = datamodule.test_dataloader()

path = "/home/work/joono/joono/joono/DV_DV.Deep/aua1cdw7/checkpoints/best-checkpoint_aug_oneshot.ckpt"
model = Wav2Vec2Facebook.load_from_checkpoint(path, args=config)



/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizat

======== Trainable Lora layers ===========
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([

In [3]:
# preds = inference(model=model, test_loader=test_loader, device='cuda:0')
preds = model.inference(test_loader=test_loader)

  0%|          | 0/1563 [00:00<?, ?it/s]


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 0

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly

In [ ]:
submit = pd.read_csv('/home/work/joono/joono/dataset/sample_submission.csv')

max_thres = 0.999
min_thres = 0.001

for i in tqdm(range(len(preds))):
    if      preds[i][0] > max_thres : submit.iloc[i, 1] = 1
    elif    preds[i][0] < min_thres : submit.iloc[i, 1] = 0 
    else                            : submit.iloc[i, 1] = preds[i][0]
    if      preds[i][1] > max_thres : submit.iloc[i, 2] = 1
    elif    preds[i][1] < min_thres : submit.iloc[i, 2] = 0 
    else                            : submit.iloc[i, 2] = preds[i][1]

submit[1000:1050]

In [ ]:
submit.to_csv('joono_wav2vec2_lora_train_test_align_test_submit3.csv', index=False)